In [134]:
from IPython.core.display import display, HTML
from h5py import File
import pandas as pd
import numpy as np
from pathlib import Path
import os
root_path = Path(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
print(root_path)
os.chdir(str(root_path))
from FIBI.data.analyse_cpp_results.tsp.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class

C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche


C:\Users\robin\AppData\Local\Temp\ipykernel_27508\1102566859.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Load data

In [135]:
data = []
with File("data/table1_exp.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data.append(dico_value)
df = pd.DataFrame(data)


In [136]:
display(df.head())

,key_dataset,SEED_POINTS,SEED_ASSIGN,NUM_CLUST,NUM_POINTS,IMPR_CLASS,IT_ORDER,init_cost,final_cost,num_iter,num_iter_glob,duration
0,"100,0,6225993,32000,0,32000,1,0",32000,32000,6225993,100,BICL,CURR,5131.130612,838.644756,106,524488,0.017645
1,"100,0,6225993,32001,0,32001,1,0",32001,32001,6225993,100,BICL,CURR,5943.732122,774.078440,107,529436,0.023554
2,"100,0,6225993,32002,0,32002,1,0",32002,32002,6225993,100,BICL,CURR,5150.832841,850.094436,108,534384,0.018902
3,"100,0,6225993,32003,0,32003,1,0",32003,32003,6225993,100,BICL,CURR,5564.167451,829.280319,100,494800,0.016120
4,"100,0,6225993,32004,0,32004,1,0",32004,32004,6225993,100,BICL,CURR,5616.584851,816.079192,102,504696,0.017493


## Verify that we have all of the required results

In [92]:
def check_print(cond:bool):
    if cond:
        print("✅")
    else:
        print("❌")

Number of points:

Expected: [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]

Found:

In [137]:
expected = [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
num_points = list(df[str(DataCategory.NUM_POINTS)].unique())
num_points.sort()
print(num_points)
check_print(set(num_points) == set(expected))

[20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]
✅


Number of different point and assign seed per number of point x algorithm

Expected:

- for n in [20, 30, ..., 150] : 1000
- for n in [200, 250, ..., 500] : 100
- for n in [500, 600, ..., 1000] : 100

Found:

In [138]:
groups_counts = df.groupby([str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)]).count().reset_index()
num_points = [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
expected = {n: 1000 if n<=150 else 100 for n in num_points}
cond = True

for n in expected:
    for impr_class in corresp_impr_class.values():
        df_tmp = groups_counts.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_POINTS)] == expected[n]
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_ASSIGN)] == expected[n]
check_print(cond)

✅


## Statistics

### Computed statistics

In [139]:
df_table1_src = df[[
    str(DataCategory.NUM_POINTS),
    str(DataCategory.IMPR_CLASS),
    str(DataCategory.final_cost),
    str(DataCategory.num_iter),
    str(DataCategory.num_iter_glob),
    str(DataCategory.duration),
    ]]
df_table1 = df_table1_src.groupby([str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)]).mean().reset_index()
df_table1 = df_table1.query(f"{str(DataCategory.IMPR_CLASS)} != 'BICL'")
df_table1.sort_values(by=[str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)][::-1], inplace=True)
df_table1.reset_index(drop=True, inplace=True)
df_table1_str = df_table1.to_html(formatters={
    str(DataCategory.NUM_POINTS): '{:d}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost): '{:.2f}'.format,
    str(DataCategory.num_iter): '{:.0f}'.format,
    str(DataCategory.num_iter_glob): '{:.0f}'.format,
    str(DataCategory.duration): '{:.4e}'.format,
})
display(HTML(df_table1_str))

,NUM_POINTS,IMPR_CLASS,final_cost,num_iter,num_iter_glob,duration
0,20,BINL,393.93,16,1887,1.6019e-04
1,30,BINL,474.50,25,6393,4.4794e-04
2,40,BINL,543.82,35,15039,9.2769e-04
3,50,BINL,605.06,46,29351,2.0379e-03
4,60,BINL,656.39,57,50190,2.9056e-03
5,70,BINL,707.80,67,78281,4.2463e-03
6,80,BINL,755.42,79,115358,6.8088e-03
7,90,BINL,796.94,90,163025,9.8987e-03
8,100,BINL,838.06,101,222061,1.3415e-02
9,110,BINL,877.65,113,293882,1.7658e-02


### Reference values

Verification manuelle algorithme par algorithme:

In [42]:
df_table1_ref_src = pd.read_csv("data/table1.csv")
# display(df_table1_ref_src.head())

In [53]:

columns = [c for c in df_table1_ref_src.columns if "percent" not in c and "NUM_POINTS" not in c]
algorithms = set([c.split("-")[1] for c in columns])
new_columns = set([c.split("-")[0] for c in columns])
data = []
for n in df_table1_ref_src[str(DataCategory.NUM_POINTS)].unique():
    n = int(n)
    for algo in algorithms:
        dico = {
            str(DataCategory.NUM_POINTS): n,
            str(DataCategory.IMPR_CLASS): algo,
        }
        for col in new_columns:
            dico[col] = df_table1_ref_src.query(f"{str(DataCategory.NUM_POINTS)} == {n}")[f"{col}-{algo}"].to_numpy()[0]
        data.append(dico)
df_table1_ref = pd.DataFrame(data)
df_table1_ref.sort_values(by=[str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)][::-1], inplace=True)
df_table1_ref.reset_index(drop=True,inplace=True)
df_table1_ref = df_table1_ref[["NUM_POINTS","IMPR_CLASS","final_cost","num_iter","CPU_time"]]
pd.set_option('display.max_rows', 1000)
display(df_table1_ref)

,NUM_POINTS,IMPR_CLASS,final_cost,num_iter,CPU_time
0,20,BINL,394.21,16,0.00
1,30,BINL,473.86,26,0.00
2,40,BINL,541.93,36,0.01
3,50,BINL,603.50,46,0.02
4,60,BINL,654.69,58,0.04
5,70,BINL,704.35,68,0.07
6,80,BINL,750.91,79,0.11
7,90,BINL,796.40,91,0.16
8,100,BINL,836.68,102,0.22
9,110,BINL,873.51,114,0.30


✅ok, same values ❌, different values

|Algorithme \ Metrique| final_cost | num_iter | duration (s)|
|:---|:---:|:---:|:---:|
|BINL|✅|✅|❌
|FICL|✅|✅|❌
|FINL|✅|✅|❌

In [154]:
computed = df_table1.copy()
reference = df_table1_ref.copy()
reference.rename(columns={"CPU_time":"duration"}, inplace=True)

computed = computed[["NUM_POINTS","IMPR_CLASS","final_cost","num_iter","duration"]]

diff = []
for n in reference[str(DataCategory.NUM_POINTS)].unique():
    for impr_class in reference[str(DataCategory.IMPR_CLASS)].unique():
        df_sel_ref = reference.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        df_sel_comp = computed.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        diff.append(
            {
                str(DataCategory.NUM_POINTS):n,
                str(DataCategory.IMPR_CLASS):impr_class, 
                str(DataCategory.final_cost)+"_ref":df_sel_ref.iloc[0]["final_cost"], 
                str(DataCategory.final_cost)+"_comp":df_sel_comp.iloc[0]["final_cost"], 
                str(DataCategory.final_cost)+"_diff":df_sel_comp.iloc[0]["final_cost"]-df_sel_ref.iloc[0]["final_cost"], 
                str(DataCategory.num_iter)+"_ref":df_sel_ref.iloc[0]["num_iter"], 
                str(DataCategory.num_iter)+"_comp":df_sel_comp.iloc[0]["num_iter"], 
                str(DataCategory.num_iter)+"_diff":df_sel_comp.iloc[0]["num_iter"]-df_sel_ref.iloc[0]["num_iter"], 
                str(DataCategory.duration)+"_ref":df_sel_ref.iloc[0]["duration"], 
                str(DataCategory.duration)+"_comp":df_sel_comp.iloc[0]["duration"],
                str(DataCategory.duration)+"_diff":df_sel_comp.iloc[0]["duration"]-df_sel_ref.iloc[0]["duration"]
            }
        )
df_diff = pd.DataFrame(diff)
df_diff_str = df_diff.to_html(formatters={
    str(DataCategory.NUM_POINTS): '{:.0f}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost)+"_ref": '{:.0f}'.format,
    str(DataCategory.final_cost)+"_comp": '{:.0f}'.format,
    str(DataCategory.final_cost)+"_diff": '{:.0f}'.format,
    str(DataCategory.num_iter)+"_ref": '{:.0f}'.format,
    str(DataCategory.num_iter)+"_comp": '{:.0f}'.format,
    str(DataCategory.num_iter)+"_diff": '{:.0f}'.format,
    str(DataCategory.duration)+"_ref": '{:.4e}'.format,
    str(DataCategory.duration)+"_comp": '{:.4e}'.format,
    str(DataCategory.duration)+"_diff": '{:.4e}'.format,
})
display(HTML(df_diff_str))
for col in [str(DataCategory.final_cost),str(DataCategory.num_iter),str(DataCategory.duration)]:
    df_diff[f"{col}_diff_abs"] = df_diff[f"{col}_diff"].abs()
    description = df_diff[f"{col}_diff_abs"].describe().to_dict()
    print(f"{col}_diff_abs: {description['min']:.2e}, {description['25%']:.2e}, {description['50%']:.2e}, {description['mean']:.2e}, {description['std']:.2e}, {description['75%']:.2e}, {description['max']:.2e}")


,NUM_POINTS,IMPR_CLASS,final_cost_ref,final_cost_comp,final_cost_diff,num_iter_ref,num_iter_comp,num_iter_diff,duration_ref,duration_comp,duration_diff
0,20,BINL,394,394,-0,16,16,-0,0.0000e+00,1.6019e-04,1.6019e-04
1,20,FICL,395,395,-1,37,37,0,0.0000e+00,7.5627e-05,7.5627e-05
2,20,FINL,394,393,-1,24,22,-2,0.0000e+00,7.7259e-05,7.7259e-05
3,30,BINL,474,474,1,26,25,-1,0.0000e+00,4.4794e-04,4.4794e-04
4,30,FICL,475,475,0,69,70,1,0.0000e+00,2.9566e-04,2.9566e-04
5,30,FINL,473,474,1,41,38,-3,0.0000e+00,1.6153e-04,1.6153e-04
6,40,BINL,542,544,2,36,35,-1,1.0000e-02,9.2769e-04,-9.0723e-03
7,40,FICL,541,541,1,107,107,0,1.0000e-02,7.8346e-04,-9.2165e-03
8,40,FINL,539,541,3,58,55,-3,0.0000e+00,2.7268e-04,2.7268e-04
9,50,BINL,604,605,2,46,46,-0,2.0000e-02,2.0379e-03,-1.7962e-02


final_cost_diff_abs: 4.11e-02, 1.33e+00, 2.76e+00, 3.48e+00, 2.87e+00, 5.29e+00, 1.05e+01
num_iter_diff_abs: 4.00e-02, 1.25e+00, 4.13e+00, 2.45e+01, 4.85e+01, 1.10e+01, 2.03e+02
duration_diff_abs: 7.56e-05, 2.02e-02, 3.43e-01, 2.28e+01, 6.07e+01, 5.36e+00, 3.54e+02
